# Оглавление
1. [Подготовка данных](#Шаг_1)
2. [Обучение моделей](#Шаг_2)
2. [Анализ моделей](#Шаг_3)

<a name="Шаг_1"></a>

# 1. Подготовка данных

In [99]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgb
from time import process_time

In [100]:
df = pd.read_csv('autos.csv')

In [101]:
df.shape

(354369, 16)

In [102]:
df.columns

Index(['DateCrawled', 'Price', 'VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired', 'DateCreated', 'NumberOfPictures', 'PostalCode',
       'LastSeen'],
      dtype='object')

In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [104]:
df = df.drop(['DateCrawled'], axis = 1)

In [105]:
df = df.drop(['RegistrationMonth'], axis = 1)

In [106]:
df = df.drop(['DateCreated'], axis = 1)

In [107]:
df = df.drop(['PostalCode'], axis = 1)

In [108]:
df = df.drop(['NumberOfPictures'], axis = 1)

In [109]:
df = df.drop(['LastSeen'], axis = 1)

DateCrawle, RegistrationMonth, DateCreated, PostalCode, (NumberOfPictures - все нули), LastSeen не интересны

In [110]:
df.VehicleType.value_counts()

sedan          91457
small          79831
wagon          65166
bus            28775
convertible    20203
coupe          16163
suv            11996
other           3288
Name: VehicleType, dtype: int64

In [111]:
df.VehicleType = df.VehicleType.fillna('other')

Пропуски в "тип автомобильного кузова" заменил на other. Считаю оптимальным вариантом. ТАк же рассматривал вариант "неизвестно" или удалить

In [112]:
df.Gearbox.value_counts()

manual    268251
auto       66285
Name: Gearbox, dtype: int64

Пропуски в "тип коробки передач" заменил на manual так как максимальное число.

In [113]:
df.Gearbox = df.Gearbox.fillna('manual')

In [114]:
df.Model.value_counts()

golf                  29232
other                 24421
3er                   19761
polo                  13066
corsa                 12570
                      ...  
i3                        8
rangerover                4
serie_3                   4
range_rover_evoque        2
serie_1                   2
Name: Model, Length: 250, dtype: int64

In [115]:
df.Model = df.Model.fillna('other')

Пропуски в "модель автомобиля" заменил на other по аналогии с "тип автомобильного кузова"

In [116]:
df.FuelType.value_counts()

petrol      216352
gasoline     98720
lpg           5310
cng            565
hybrid         233
other          204
electric        90
Name: FuelType, dtype: int64

In [117]:
df.FuelType = df.FuelType.fillna('petrol')

Пропуски в "тип топлива" заменил на petrol. 1) petrol - большинство с отрывом. 2) Считаю возможным вариант того, что поскольку petrol самый распространенный, можо считать установку по умолчанию. (просто не сочли нужным выбрать в анкете.)

In [118]:
df.NotRepaired.value_counts()

no     247161
yes     36054
Name: NotRepaired, dtype: int64

In [119]:
df.NotRepaired = df.NotRepaired.fillna('no')

Пропуски в "была машина в ремонте или нет" заменил на no. 1) no большинство 2) Отсутствие ремонта можно просто не помечать. Нет и нет

Итого с пропусками все. Далее к значениям.

In [120]:
df.Price.describe()

count    354369.000000
mean       4416.656776
std        4514.158514
min           0.000000
25%        1050.000000
50%        2700.000000
75%        6400.000000
max       20000.000000
Name: Price, dtype: float64

Цену меньше 100 считаю не приемлимой (99 - типо акция / все по 99,99 $/ for sale)

In [121]:
df = df[df.Price >= 99]

In [122]:
df.Price.describe()

count    341164.000000
mean       4587.426566
std        4514.836512
min          99.000000
25%        1200.000000
50%        2900.000000
75%        6500.000000
max       20000.000000
Name: Price, dtype: float64

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 341164 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             341164 non-null  int64 
 1   VehicleType       341164 non-null  object
 2   RegistrationYear  341164 non-null  int64 
 3   Gearbox           341164 non-null  object
 4   Power             341164 non-null  int64 
 5   Model             341164 non-null  object
 6   Kilometer         341164 non-null  int64 
 7   FuelType          341164 non-null  object
 8   Brand             341164 non-null  object
 9   NotRepaired       341164 non-null  object
dtypes: int64(4), object(6)
memory usage: 28.6+ MB


In [124]:
df.VehicleType.value_counts()

sedan          89094
small          77494
wagon          63474
other          35702
bus            28266
convertible    19817
coupe          15555
suv            11762
Name: VehicleType, dtype: int64

Нормально

In [125]:
df.RegistrationYear.describe()

count    341164.000000
mean       2003.985646
std          71.315286
min        1000.000000
25%        1999.000000
50%        2003.000000
75%        2008.000000
max        9999.000000
Name: RegistrationYear, dtype: float64

Минимальный 1000, маскимальный 9999. Надо убирать

In [126]:
(pd.DataFrame(data=df.RegistrationYear.value_counts())).sort_index().head(60)

,RegistrationYear
1000,22
1001,1
1039,1
1111,1
1234,4
1255,1
1300,2
1400,1
1500,3
1600,2


Нижний 1960. На автору нашел Mercedes-Benz Simplex 1904 года. Но это не наш случай. цены не те за раритет.

In [127]:
(pd.DataFrame(data=df.RegistrationYear.value_counts())).sort_index().tail(40)

,RegistrationYear
2011,9814
2012,7109
2013,4078
2014,2494
2015,1168
2016,8851
2017,9835
2018,3766
2019,16
2066,1


Верхний потлок 2019

In [128]:
df = df[(df.RegistrationYear >= 1960) & (df.RegistrationYear <= 2019)] 

In [129]:
df.Gearbox.value_counts()

manual    276139
auto       64649
Name: Gearbox, dtype: int64

Нормально

In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 340788 entries, 0 to 354368
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Price             340788 non-null  int64 
 1   VehicleType       340788 non-null  object
 2   RegistrationYear  340788 non-null  int64 
 3   Gearbox           340788 non-null  object
 4   Power             340788 non-null  int64 
 5   Model             340788 non-null  object
 6   Kilometer         340788 non-null  int64 
 7   FuelType          340788 non-null  object
 8   Brand             340788 non-null  object
 9   NotRepaired       340788 non-null  object
dtypes: int64(4), object(6)
memory usage: 28.6+ MB


In [131]:
df.Power.value_counts()

0        34914
75       23189
60       15360
150      14105
101      12925
         ...  
6226         1
337          1
1105         1
1199         1
13636        1
Name: Power, Length: 696, dtype: int64

1000 Беру за максимум

In [132]:
(pd.DataFrame(data=df.Power.value_counts())).sort_index().head(35)

,Power
0,34914
1,23
2,9
3,8
4,30
5,99
6,11
7,10
8,6
9,4


ВАЗ 11113 - 32 лс. верхняя граница


In [133]:
df = df[df.Power <= 1000] 

In [134]:
df['Power'].median()

105.0

In [135]:
df.loc[df['Power'] == 0, 'Power'] = 105

In [136]:
df = df[(df.Power >= 40) & (df.Power <= 1000)] 

In [137]:
df.Model.value_counts()

other                 39652
golf                  28054
3er                   19053
polo                  12470
corsa                 12011
                      ...  
i3                        8
rangerover                4
serie_3                   3
range_rover_evoque        2
serie_1                   1
Name: Model, Length: 250, dtype: int64

Норма

In [138]:
df.Kilometer.describe()

count    339234.000000
mean     128779.176026
std       36928.970863
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: Kilometer, dtype: float64

норма

In [139]:
df.FuelType.value_counts()

petrol      236419
gasoline     96728
lpg           5150
cng            546
hybrid         226
other          116
electric        49
Name: FuelType, dtype: int64

норма

In [140]:
df.Brand.value_counts()

volkswagen        73637
opel              38060
bmw               35499
mercedes_benz     31146
audi              28378
ford              24152
renault           17149
peugeot           10696
fiat               9016
seat               6660
mazda              5431
skoda              5414
smart              5125
citroen            4895
nissan             4769
toyota             4520
hyundai            3513
mini               3155
volvo              3101
mitsubishi         2903
honda              2699
kia                2394
sonstige_autos     2373
suzuki             2241
alfa_romeo         2205
chevrolet          1685
chrysler           1347
dacia               887
daihatsu            758
subaru              719
porsche             701
jeep                640
land_rover          532
daewoo              527
saab                512
jaguar              486
rover               455
lancia              447
lada                213
trabant             194
Name: Brand, dtype: int64

норма

In [141]:
df.NotRepaired.value_counts()

no     305924
yes     33310
Name: NotRepaired, dtype: int64

In [142]:
df.loc[df['NotRepaired'] == 'no', 'NotRepaired'] = 0

In [143]:
df.loc[df['NotRepaired'] == 'yes', 'NotRepaired'] = 1

Поменял на цифры.

<a name="Шаг_2"></a>

# Обучение моделей

In [144]:
features = df.drop(['Price'], axis = 1)

In [145]:
target = df['Price']

In [146]:
encoder = OrdinalEncoder()

In [147]:
df_encoder = features.drop((['RegistrationYear', 'Power', 'Kilometer', 'NotRepaired']), axis = 1)

In [148]:
df_encoder = df_encoder.reset_index()

In [149]:
df_encoder = df_encoder.drop(['index'], axis = 1)

In [150]:
features_encoder = pd.DataFrame(encoder.fit_transform(df_encoder),columns=df_encoder.columns)

In [151]:
features = features.reset_index()

In [152]:
features = features.drop((['index']), axis = 1)

In [153]:
features['VehicleType'] = features_encoder['VehicleType']

In [154]:
features['Gearbox'] = features_encoder['Gearbox']

In [155]:
features['Model'] = features_encoder['Model']

In [156]:
features['FuelType'] = features_encoder['FuelType']

In [157]:
features['Brand'] = features_encoder['Brand']

In [158]:
scaler = StandardScaler()

In [159]:
scaler.fit(features)

StandardScaler()

In [160]:
features = pd.DataFrame(data = scaler.transform(features),columns=features.columns)

In [161]:
features

,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,FuelType,Brand,NotRepaired
0,-0.565327,-1.466537,0.484060,-0.274359,0.060290,0.574640,0.652573,1.290329,-0.329975
1,-1.055531,1.095362,0.484060,1.368034,0.774366,-0.102336,-1.552757,-1.493510,3.030536
2,0.905285,0.099068,-2.065859,0.846333,0.074572,-0.102336,-1.552757,-0.515404,-0.329975
3,0.415081,-0.327915,0.484060,-0.854027,0.060290,0.574640,0.652573,1.290329,-0.329975
4,0.415081,0.668379,0.484060,-0.969961,-0.153933,-1.050103,-1.552757,0.763657,-0.329975
...,...,...,...,...,...,...,...,...,...
339229,-2.035939,-0.470243,0.484060,-0.274359,1.959732,0.574640,0.652573,0.236985,-0.329975
339230,-0.565327,0.241396,0.484060,-0.274359,0.774366,-2.945637,0.652573,0.914134,-0.329975
339231,-1.545735,-0.470243,-2.065859,-0.351648,-0.082525,-0.102336,0.652573,0.838896,-0.329975
339232,-2.035939,-1.039554,0.484060,-0.332326,1.602694,0.574640,-1.552757,1.290329,-0.329975


In [162]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345)

#### array

In [163]:
features_train = np.array(features_train)

In [164]:
features_valid = np.array(features_valid)

In [165]:
target_train = np.array(target_train)

In [166]:
target_valid = np.array(target_valid)

#### LinearRegression

In [167]:
start = process_time()

In [168]:
model_LR = LinearRegression()

In [169]:
model_LR.fit(features_train, target_train)

LinearRegression()

In [170]:
predict_LR = model_LR.predict(features_valid)

In [171]:
rsme_LR = mean_squared_error(target_valid, predict_LR) ** 0.5

In [172]:
stop = process_time() 

In [173]:
LR_time = stop-start

#### RandomForestRegressor

In [174]:
start = process_time()

In [175]:
model_RFR = RandomForestRegressor(n_estimators = 70, max_depth=18, random_state = 12345)

In [176]:
model_RFR.fit(features_train, target_train)   

RandomForestRegressor(max_depth=18, n_estimators=70, random_state=12345)

In [177]:
predict_RFR = model_RFR.predict(features_valid)

In [178]:
rsme_RFR = mean_squared_error(target_valid, predict_RFR) ** 0.5

In [179]:
stop = process_time() 

In [180]:
RFR_time =  stop-start

#### LightGMB 

In [181]:
start = process_time()

In [182]:
model_LGMB = lgb.LGBMRegressor(max_depth = 20, num_leaves = 202,  random_state = 12345)

In [183]:
model_LGMB.fit(features_train, target_train)

LGBMRegressor(max_depth=20, num_leaves=202, random_state=12345)

In [184]:
predict_LGMB = model_LGMB.predict(features_valid)

In [185]:
rsme_LGMB = mean_squared_error(target_valid, predict_LGMB) ** 0.5

In [186]:
stop = process_time() 

In [187]:
LightGMB_time = stop-start

#### Таблица

In [188]:
table = pd.DataFrame({'name': ['LinearRegression', 'RandomForestRegressor', 'LightGMB']
                      , 'rsme': [rsme_LR, rsme_RFR, rsme_LGMB], 'time': [LR_time, RFR_time, LightGMB_time]})

<a name="Шаг_3"></a>

# Анализ моделей

In [190]:
table

,name,rsme,time
0,LinearRegression,3071.832577,0.890625
1,RandomForestRegressor,1642.402298,64.796875
2,LightGMB,1625.285639,15.890625


#### Итого 

Минимальное отклонение и минимальоне время работы (обучение и предсказания) показала модель LightGMB.

In [191]:
predict = model_LGMB.predict(features)

In [192]:
mean_squared_error(target, predict) ** 0.5

1537.5309819521444

Проверил на всем объеме!